# pretty_midi
[公式](http://craffel.github.io/pretty-midi/)  
MIDIを簡単に修正したり，情報を取得することができるライブラリ  
  
研究で使用するには，以下の引用が必要  
Colin Raffel and Daniel P. W. Ellis. Intuitive Analysis, Creation and Manipulation of MIDI Data with pretty_midi. In 15th International Conference on Music Information Retrieval Late Breaking and Demo Papers, 2014.

FluidSynthによって書き出すことができる  
インストール方法は`pyfluidsynth_install.md`に記載

## 基本的な使い方

In [1]:
import pretty_midi, wave, struct, numpy as np
from scipy.io import wavfile as spw
from IPython.display import Audio
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# MIDIデータの読み込み
midi_data = pretty_midi.PrettyMIDI('./midi/a.mid')

# テンポの経験的推定
print("Estimated tempo:", midi_data.estimate_tempo())

Estimated tempo: 224.17791898332382


In [6]:
# 12音階でどこがよく使われているかを調べ，スケールを推定

# 12音階に分けたMIDIデータ
chroma = midi_data.get_chroma()
print(chroma.shape)

# 12音階のそれぞれがどれだけ使われているかの相対量を取得
total_velocity = sum(sum(chroma))
rel_amount = [sum(semitone)/total_velocity for semitone in chroma]
for k in range(12):
    key_name = pretty_midi.note_number_to_name(k)[:-2]
    print("{:<2}".format(key_name), ":", rel_amount[k])
print(midi_data.key_signature_changes)

(12, 21800)
C  : 0.1627839844479718
C# : 0.0
D  : 0.08120311717131701
D# : 0.0
E  : 0.158110031652076
F  : 0.10397721997722975
F# : 0.0
G  : 0.12141781078591192
G# : 0.0
A  : 0.31742115829082546
A# : 0.003716422056246051
B  : 0.051370255618422025
[]


Amかな？

In [4]:
# ドラム以外のMIDIを+5トランスポーズ
for instrument in midi_data.instruments:
    if instrument.is_drum: continue
    for note in instrument.notes:
        note.pitch += 5

### MIDIファイルの生成

In [9]:
cello_c_chord = pretty_midi.PrettyMIDI()

# 空のInstrument(トラック)を取得
cello_program = pretty_midi.instrument_name_to_program("Cello")
cello = pretty_midi.Instrument(program=cello_program)

# 0[s] ~ 0.5[s] のCコードを設定
for note_name in ['C5', 'E5', 'G5']:
    note_number = pretty_midi.Note(velocity=100, pitch=note_name, start=0, end=.5)
    cello.notes.append(note)

# トラックにcelloを設定
cello_c_chord.instruments.append(cello)
cello_c_chord.write('./midi/cello-C-chord.mid')

### WAVEファイルとして書き出し
([参考](https://qiita.com/kinaonao/items/c3f2ef224878fbd232f5))

In [5]:
def pm_to_wave(pm, wave_file_name, wave_form=np.sin, fs=44100, fluid=True, sf_path=None):
    
    # サイン波によるMIDIデータのオーディオデータへの変換 [-1,1]
    if fluid:
        audio = midi_data.fluidsynth(fs, sf_path)
    else:
        audio = midi_data.synthesize(fs, wave_form)
    
    # 16bit=2byte符号付き整数に変換 [-32768  ~ 32767]
    audio = [int(x * 32767.0) for x in audio]

    # 2byteのデータとしてバイナリ化(https://techacademy.jp/magazine/19058)
    binwave = struct.pack("h" * len(audio), *audio)
    
    # waveファイルの準備
    w = wave.Wave_write(wave_file_name)

    # チャンネル数(1:Mono,2:Stereo)、サンプルサイズ(バイト)、サンプリング周期
    # フレーム数，圧縮形式('NONE'のみ)、圧縮形式の名前('not compressed'のみ)
    w.setparams((1, 2, fs, len(binwave), 'NONE', 'not compressed'))
    
    # 書き出しと終了
    w.writeframes(binwave)
    w.close()
    
    return audio

scipyでも書き出せる．こっちの方が早い([参考](http://naga-tsuzuki.sblo.jp/article/176554546.html))

In [4]:
def pm_to_wave(pm, wave_file_name, wave_form=np.sin, fs=44100, fluid=True, sf_path=None):
    
    # サイン波によるMIDIデータのオーディオデータへの変換 [-1,1]
    if fluid:
        audio = midi_data.fluidsynth(fs, sf_path)
    else:
        audio = midi_data.synthesize(fs, wave_form)
    
    # 16bit=2byte符号付き整数に変換してノーマライズ [-32768  ~ 32767]
    audio = np.array(audio * 32767.0, dtype="int16") # floatだと情報量が多くなる
    audio_stereo = np.c_[audio, audio] # ステレオ化
    spw.write(wave_file_name, fs, audio_stereo) # 書き出し
    
    return audio

In [ ]:
wave_file_name = "./data/sample.wav"
# sf_path = "./data/GeneralUser_GS_v1.471.sf2" # mac
sf_path = "../gsfont/gsfont.sf2" # ubuntu
audio = pm_to_wave(midi_data, wave_file_name, sf_path=sf_path)
print(len(audio))
plt.plot(audio)
Audio(wave_file_name)

その他の例は[ここ](https://github.com/craffel/pretty-midi/tree/master/examples)
- audio-to-MIDI
- チップチューンの作成
- コードカタログ
- オーディオのピアノロール化(librosa)

# クラス

In [20]:
def methods(obj):
    return [attr for attr in dir(obj) if not attr.startswith('_')]

### PrettyMIDI
Parameters
- midi_file=None
- resolution=220
- initial_tempo=120.0

Attributes
- instruments: `Instrument`のリスト
- key_signature_changes: `KeySignature`のリスト
- time_signature_changes: `TimeSignature`のリスト
- lyrics: `Lyrics`のリスト

In [19]:
methods(pretty_midi.PrettyMIDI())

['adjust_times',
 'estimate_beat_start',
 'estimate_tempi',
 'estimate_tempo',
 'fluidsynth',
 'get_beats',
 'get_chroma',
 'get_downbeats',
 'get_end_time',
 'get_onsets',
 'get_piano_roll',
 'get_pitch_class_histogram',
 'get_pitch_class_transition_matrix',
 'get_tempo_changes',
 'instruments',
 'key_signature_changes',
 'lyrics',
 'remove_invalid_notes',
 'resolution',
 'synthesize',
 'tick_to_time',
 'time_signature_changes',
 'time_to_tick',
 'write']

やれることをまとめる．詳しい使い方は公式を参照
- adjust_times: ある時間のMIDIイベントを別の時間のMIDIイベントに移動
- fluidsynth: fluidsynthによる`.sf2`サウンドフォントを使用した音声合成
- chroma: 全楽器の音符を12音階にまとめたもの
- onsets: 全ての音符の開始時間
- piano_roll: (128, time_steps), fsによってサンプリング周波数を指定
- pitch_class_histogram: ピッチクラスのヒストグラムを作成
- pitch_class_transition_matrix: 一定時間を超える長さの音符の音程変化?
- remove_invalid_notes: 開始時間より終了時間が前にある音符を削除
- tick: 2×resolution = 1 tick?
- write: MIDIファイルを書き出し

### Instrument
Paramters
- program: MIDIの楽器プログラム番号
- is_drum: ドラム(channel 9)であるか？
- name: 楽器名

Attributes
- program
- is_drum
- name
- notes: `Note`オブジェクトのリスト
- pitch_bends: `PitchBend`オブジェクトのリスト
- control_changes: `ControlChange`オブジェクトのリスト

In [23]:
methods(pretty_midi.Instrument(0, False, 'piano'))

['control_changes',
 'fluidsynth',
 'get_chroma',
 'get_end_time',
 'get_onsets',
 'get_piano_roll',
 'get_pitch_class_histogram',
 'get_pitch_class_transition_matrix',
 'is_drum',
 'name',
 'notes',
 'pitch_bends',
 'program',
 'remove_invalid_notes',
 'synthesize']

### Note
Paramters
- velocity: 演奏の強さ
- pitch: 音程番号
- start: 開始時間(秒)
- end: 終了時間(秒)

### PitchBend
Parameters
- pitch: ピッチベンド値[-8192~8191]
- time: 発生時間(秒)

### ControlChange
Parameters
- number: CC番号[0,127]
- value: CC値[0,127]
- time: 発生時間(秒)

### TimeSignature
Parameters
- numerator: 分子
- denominator: 分母
- time: 発生時間(秒)

### KeySignature
Parameters
- key_number: [0~11]がMajor, [12,23]がMinor
- time: 発生時間(秒)

### Lyric
Parameters
- text: 歌詞テキスト
- time: 発生時間(秒)

# 関数

In [33]:
def functions(obj):
    return [attr for attr in dir(obj) if not (attr.startswith('_') or attr[0].isupper())]

In [34]:
functions(pretty_midi)

['collections',
 'constants',
 'containers',
 'copy',
 'drum_name_to_note_number',
 'functools',
 'hz_to_note_number',
 'instrument',
 'instrument_name_to_program',
 'key_name_to_key_number',
 'key_number_to_key_name',
 'key_number_to_mode_accidentals',
 'math',
 'mido',
 'mode_accidentals_to_key_number',
 'note_name_to_number',
 'note_number_to_drum_name',
 'note_number_to_hz',
 'note_number_to_name',
 'np',
 'os',
 'pitch_bend_to_semitones',
 'pkg_resources',
 'pretty_midi',
 'program_to_instrument_class',
 'program_to_instrument_name',
 'qpm_to_bpm',
 're',
 'semitones_to_pitch_bend',
 'six',
 'utilities',
 'warnings']

できることをまとめていく
- key_number, key_name: 音程の番号と名前の相互変換
- mode_actidential, key_number: ex. シャープ3つならAが主音，AはCから数えて9だから9が返る
- qpm, bpm: 1分間にいくつ4分音符が出てくるかvs拍が出てくるか
- note_number, hz: 音程番号とヘルツ[Hz]の相互変換 69(A)⇔440 ピッタリじゃなくてもいける
- note_name, number: A4 ⇔ 69
- note_number, drum_name: ドラムパートの楽器の名前 [参照](https://www.midi.org/specifications/item/gm-level-1-sound-set)
- program, instrument_name: プログラム番号と楽器名 [参照](https://www.midi.org/specifications/item/gm-level-1-sound-set)
- program, instrument_class: プログラム名と楽器種別 [参照](https://www.midi.org/specifications/item/gm-level-1-sound-set)
- pitchbend, semitone: ピッチベンドの値が何半音に当たるか．semitone_rangeを変更可能